In [2]:
pip install -U textblob


   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   - ------------------------------------- 30.7/626.3 kB 640.0 kB/s eta 0:00:01
   -------- ------------------------------- 133.1/626.3 kB 1.6 MB/s eta 0:00:01
   -------------------------------- ------- 512.0/626.3 kB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 626.3/626.3 kB 4.4 MB/s eta 0:00:00


In [1]:
pip install selenium

   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.5 MB 3.4 MB/s eta 0:00:03
   - -------------------------------------- 0.3/9.5 MB 3.2 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/9.5 MB 5.5 MB/s eta 0:00:02
   -------- ------------------------------- 1.9/9.5 MB 11.1 MB/s eta 0:00:01
   --------------- ------------------------ 3.6/9.5 MB 16.5 MB/s eta 0:00:01
   ------------------------ --------------- 5.8/9.5 MB 21.7 MB/s eta 0:00:01
   --------------------------------- ------ 8.0/9.5 MB 25.5 MB/s eta 0:00:01
   ---------------------------------------  9.5/9.5 MB 27.4 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 25.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/467.7 kB ? eta -:--:--
   --------------------------------------- 467.7/467.7 kB 28.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/182.0 kB ? eta -:--:--
   ----------------

In [11]:
import time
import pandas as pd
import re, string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from textblob import TextBlob
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [18]:
data=[]

with Chrome(executable_path=r'../Processamento_linguagem_natural/chromedriver-win64/chromedriver.exe') as driver:
    wait = WebDriverWait(driver,15)
    driver.get("https://www.youtube.com/watch?v=KJzBxJ8ExRk")

    for item in range(30): 
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(15)

    for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):
        data.append(comment.text)

df = pd.DataFrame(data, columns=['comment'])
df.head()

TypeError: WebDriver.__init__() got an unexpected keyword argument 'executable_path'

In [ ]:
sw = stopwords.words('portuguese')

def clean_text(text):
    text = text.lower()
    text = re.sub('@', '', text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r"[^a-zA-Z ]+", "", text)
    
    #Tokenize the data
    text = nltk.word_tokenize(text)
    #Remove stopwords
    text = [w for w in text if w not in sw]
    return text

In [ ]:
df['comment'] = df['comment'].apply(lambda x: clean_text(x))

lemmatizer = WordNetLemmatizer()

def lem(text):
    text = [lemmatizer.lemmatize(t) for t in text]
    text = [lemmatizer.lemmatize(t, 'v') for t in text]
    return text

df['comment'] = df['comment'].apply(lambda x: lem(x))

In [ ]:
# GRAFICO LISTA COM TODAS AS PALAVRAS
all_words=[]

for i in range(len(df)):
    all_words = all_words + df['comment'][i]      
nlp_words = nltk.FreqDist(all_words)
plot1 = nlp_words.plot(20, color='salmon', title='Frequência de palavras')

In [ ]:
# GRAFICO  de Bigramas
bigrm = list(nltk.bigrams(all_words))
words_2 = nltk.FreqDist(bigrm)
words_2.plot(20, color='salmon', title='Frequência de bigramas')